In [1]:
# Q1A

In [2]:
hs = 0.6
ds = 0.4
A_hs = 0.3
A_ds = 0.2

#P(hs) ? given A grade
#hs_A = A_hs * hs / TP

hs_A = (A_hs * hs) / (A_hs*hs + A_ds*ds)
hs_A

0.6923076923076923

In [3]:
# Q1B

In [5]:
dis = 0.01
tp = 0.99
fp = 0.02
#P(tp?)

ans = dis*tp / (dis*tp + (1-dis)*fp)
ans

0.3333333333333333

In [6]:
#Q2

In [19]:
import pandas as pd

data = {
    'age': ['<=30', '<=30', '31…40', '>40', '>40', '>40', '31…40', '<=30', '<=30', '>40',
            '<=30', '31…40', '31…40', '>40'],
    'income': ['high', 'fair', 'high', 'excellent', 'high', 'fair', 'medium', 'fair', 'low', 'fair',
               'excellent', 'low', 'excellent', 'medium'],
    'student': ['no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes',
                'no', 'yes', 'yes', 'no'],
    'credit_rating': ['fair', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'yes',
                      'no', 'yes', 'yes', 'no'],
    'buys_computer': ['no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes',
                      'no', 'yes', 'yes', 'no']
}

df = pd.DataFrame(data)
df

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,fair,no,no,no
2,31…40,high,no,no,no
3,>40,excellent,no,no,no
4,>40,high,yes,no,yes
5,>40,fair,yes,yes,yes
6,31…40,medium,no,no,yes
7,<=30,fair,yes,yes,yes
8,<=30,low,yes,yes,no
9,>40,fair,yes,yes,yes


In [35]:
total_count = len(df)
class_counts = df['buys_computer'].value_counts()
prior_probs = class_counts / total_count
prior_probs

buys_computer
no     0.5
yes    0.5
Name: count, dtype: float64

In [36]:
class_counts

buys_computer
no     7
yes    7
Name: count, dtype: int64

In [37]:
subset

,age,income,student,credit_rating,buys_computer
4,>40,high,yes,no,yes
5,>40,fair,yes,yes,yes
6,31…40,medium,no,no,yes
7,<=30,fair,yes,yes,yes
9,>40,fair,yes,yes,yes
11,31…40,low,yes,yes,yes
12,31…40,excellent,yes,yes,yes


In [30]:
features = ['age', 'income', 'student', 'credit_rating']
likelihoods = {}

for feature in features:
    for class_value in class_counts.index:
        subset = df[df['buys_computer'] == class_value]
        feature_counts = subset[feature].value_counts()
        feature_likelihoods = (feature_counts + 1) / (len(subset) + len(df[feature].unique())) # Laplace smoothing
        likelihoods[(feature, class_value)] = feature_likelihoods

likelihoods #Featurs given Y=Yes and Y=No

{('age',
  'no'): age
 <=30     0.5
 >40      0.3
 31…40    0.2
 Name: count, dtype: float64,
 ('age',
  'yes'): age
 >40      0.4
 31…40    0.4
 <=30     0.2
 Name: count, dtype: float64,
 ('income',
  'no'): income
 high         0.250000
 excellent    0.250000
 fair         0.166667
 low          0.166667
 medium       0.166667
 Name: count, dtype: float64,
 ('income',
  'yes'): income
 fair         0.333333
 high         0.166667
 medium       0.166667
 low          0.166667
 excellent    0.166667
 Name: count, dtype: float64,
 ('student',
  'no'): student
 no     0.777778
 yes    0.222222
 Name: count, dtype: float64,
 ('student',
  'yes'): student
 yes    0.777778
 no     0.222222
 Name: count, dtype: float64,
 ('credit_rating',
  'no'): credit_rating
 no      0.6
 fair    0.2
 yes     0.2
 Name: count, dtype: float64,
 ('credit_rating',
  'yes'): credit_rating
 yes    0.6
 no     0.3
 Name: count, dtype: float64}

In [31]:
def predict(features_dict):
    class_probs = {}
    for class_value in class_counts.index:
        prob = prior_probs[class_value]
        for feature, value in features_dict.items():
            prob = prob * likelihoods.get((feature, class_value), {}).get(value, 1 / (len(df) + len(df[feature].unique())))
        class_probs[class_value] = prob

    return max(class_probs, key=class_probs.get)

In [33]:
predictions = []
for _, row in df.iterrows():
    sample = row[['age', 'income', 'student', 'credit_rating']].to_dict()
    pred = predict(sample)
    predictions.append(pred)

df['predicted_buys_computer'] = predictions
print(df[['buys_computer', 'predicted_buys_computer']])

   buys_computer predicted_buys_computer
0             no                      no
1             no                      no
2             no                      no
3             no                      no
4            yes                     yes
5            yes                     yes
6            yes                      no
7            yes                     yes
8             no                     yes
9            yes                     yes
10            no                      no
11           yes                     yes
12           yes                     yes
13            no                      no


In [38]:
# Q3

In [46]:
import csv
import math
from collections import defaultdict

def load_dataset(filename):
    """Load the dataset from a CSV file."""
    dataset = []
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            dataset.append((row['Text'], row['Tag']))
    return dataset

def naive_bayes_train(dataset):
    """Train the Naive Bayes classifier."""
    classes = set(row[1] for row in dataset)
    class_probabilities = defaultdict(int)
    word_probabilities = defaultdict(lambda: defaultdict(int))

    for text, tag in dataset:
        class_probabilities[tag] += 1
        words = text.split()
        for word in words:
            word_probabilities[tag][word] += 1

    for tag in classes:
        class_probabilities[tag] /= len(dataset)
        for word in word_probabilities[tag]:
            word_probabilities[tag][word] /= sum(word_probabilities[tag].values())

    return class_probabilities, word_probabilities

def naive_bayes_classify(text, class_probabilities, word_probabilities):
    """Classify a text using the Naive Bayes classifier."""
    words = text.split()
    scores = defaultdict(int)
    for tag in class_probabilities:
        scores[tag] = math.log(class_probabilities[tag])
        for word in words:
            if word in word_probabilities[tag]:
                scores[tag] += math.log(word_probabilities[tag][word])
    return max(scores, key=scores.get)

def evaluate_model(dataset, class_probabilities, word_probabilities):
    """Evaluate the model using accuracy, precision, and recall."""
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    for text, tag in dataset:
        predicted_tag = naive_bayes_classify(text, class_probabilities, word_probabilities)
        if predicted_tag == tag:
            if tag == 'Sports':
                true_positives += 1
            else:
                true_negatives += 1
        else:
            if predicted_tag == 'Sports':
                false_positives += 1
            else:
                false_negatives += 1

    accuracy = (true_positives + true_negatives) / len(dataset)
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0

    return accuracy, precision, recall

# Load the dataset
dataset = load_dataset('dataset.csv')

# Train the Naive Bayes classifier
class_probabilities, word_probabilities = naive_bayes_train(dataset)

# Evaluate the model
accuracy, precision, recall = evaluate_model(dataset, class_probabilities, word_probabilities)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

# Classify a new text
new_text = "A very close game"
predicted_tag = naive_bayes_classify(new_text, class_probabilities, word_probabilities)
print(f'The text "{new_text}" belongs to the tag "{predicted_tag}"')

Accuracy: 0.00
Precision: 0.00
Recall: 0.00
The text "A very close game" belongs to the tag "Not sports"
